In [2]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, root_dir, split, transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform        
        self.images = []
        self.labels_info = []
        
        # Load image paths from the text file corresponding to the split
        with open(os.path.join(root_dir, f'{split}.txt'), 'r') as file:
            for line in file:
                image_relative_path = line.strip()
                image_path = os.path.join(root_dir, 'images', image_relative_path)
                label_path = os.path.join(root_dir, 'labels', os.path.splitext(image_relative_path)[0] + '.txt')
                self.images.append(image_path)
                self.labels_info.append(label_path)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label_path = self.labels_info[idx]
        
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, _ = image.shape

        boxes = []
        class_labels = []
        with open(label_path, 'r') as file:
            for line in file:
                class_label, x_center, y_center, w, h = map(float, line.strip().split())
                boxes.append([x_center, y_center, w, h])
                class_labels.append(int(class_label))
        
        boxes = np.array(boxes)
        class_labels = np.array(class_labels)
        
        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes, class_labels=class_labels)
            image = transformed['image']
            boxes = transformed['bboxes']
            class_labels = transformed['class_labels']
        
        boxes = torch.tensor(boxes)
        class_labels = torch.tensor(class_labels)
        labels = torch.cat((class_labels.unsqueeze(1).float(), boxes), dim=1)
        
        return image, labels

# Augmentation transformations
transform = A.Compose([
    A.RandomResizedCrop(height=450, width=450, scale=(0.8, 1.0), p=1.0), # random Resize and crop
    A.HorizontalFlip(p=0.5), # Horizontal Flip
    A.Blur(p=0.1), # gaussian blur
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.2), # color jitter
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.2), # grid distortion
    A.CLAHE(p=0.1), # Constant limited adaptive histogram equalization
    A.RandomBrightnessContrast(p=0.2), # Random brightness and contrast
    A.Affine(scale=(0.8, 1.2), translate_percent=(-0.1, 0.1), rotate=(-10, 10), shear=(-5, 5), p=0.5), # Affine transformation
    A.Resize(width=640, height=640, p=1.0), # Resize 
    ToTensorV2(),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))



# Root directory
root_dir = r'euroshop\euroshop_dataset'

# Train and validation datasets
train_dataset = CustomDataset(root_dir, split='train', transform=transform)
val_dataset = CustomDataset(root_dir, split='val', transform=None)
test_dataset = CustomDataset(root_dir, split='test', transform=None)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

In [3]:
from ultralytics import YOLO


# Load the YOLOv8n model
model = YOLO('yolov8n.pt')

# Set the dataloaders
model.train_loader = train_loader
model.val_loader = val_loader
model.test_loader = test_loader

# Train the model
model.train(data='euroshop.yaml', epochs=50, batch=16 , optimizer='RAdam', lr0=0.001, close_mosaic=0)


New https://pypi.org/project/ultralytics/8.2.31 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.15  Python-3.10.14 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=euroshop.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train56, exist_ok=False, pretrained=True, optimizer=RAdam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,

train: Scanning C:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\labels... 6517 images, 3 backgrounds, 0 corrupt: 100%|██████████| 6517/6517 [00:05<00:00, 1217.68it/s]


train: New cache created: C:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


C:\Users\ilias\AppData\Roaming\Python\Python310\site-packages\albumentations\core\composition.py:144: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
val: Scanning C:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\labels... 1862 images, 2 backgrounds, 0 corrupt: 100%|██████████| 1862/1862 [00:01<00:00, 992.89it/s] 


val: New cache created: C:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\labels.cache
Plotting labels to runs\detect\train56\labels.jpg... 
optimizer: RAdam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train56
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1862      19165      0.873      0.911      0.931      0.822



50 epochs completed in 0.507 hours.
Optimizer stripped from runs\detect\train56\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train56\weights\best.pt, 6.2MB

Validating runs\detect\train56\weights\best.pt...
Ultralytics YOLOv8.2.15  Python-3.10.14 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 59/59 [00:09<00:00,  6.52it/s]


                   all       1862      19165      0.878      0.914      0.935      0.824
                   bag       1862         32          1      0.999      0.995      0.811
                  cart       1862         47      0.609      0.766      0.727      0.646
                  hand       1862       5035      0.877      0.915      0.958      0.792
                  head       1862       1811      0.914      0.903      0.972      0.902
                person       1862       5684      0.913      0.959      0.982      0.926
               product       1862       6556      0.955      0.941      0.974      0.865
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train56


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001DDDD3AC490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

In [4]:
# Perform evaluation on the test dataset

test_results = model.val(split='test', plots=True)
print(test_results)

Ultralytics YOLOv8.2.15  Python-3.10.14 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\labels... 932 images, 1 backgrounds, 0 corrupt: 100%|██████████| 932/932 [00:00<00:00, 1126.99it/s]


val: New cache created: C:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 59/59 [00:07<00:00,  8.33it/s]


                   all        932       9761      0.853      0.867      0.908      0.809
                   bag        932         21      0.976      0.952      0.989      0.829
                  cart        932         14      0.429      0.571      0.566      0.514
                  hand        932       2635      0.897       0.89      0.955      0.796
                  head        932        943      0.913      0.892      0.971      0.908
                person        932       2958       0.93      0.951      0.984       0.93
               product        932       3190       0.97      0.945       0.98      0.878
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train562
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001DDDDAF8B20>
curves: ['Precisi

In [5]:
# Visualize the results

test_r = model(source='euroshop/euroshop_dataset/test_10.txt', project='test_results\model3', save=True)

for test_r in test_r:
    boxes = test_r.boxes
    masks = test_r.masks
    keypoints = test_r.keypoints
    probs = test_r.probs
    obb = test_r.obb
    #test_r.show()


image 1/10 c:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\images\192.168.168.102_09_58_07_01c216e6-d39c-4c5b-b092-72179f22c515_474.jpg: 384x640 2 hands, 2 persons, 5 products, 77.0ms
image 2/10 c:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\images\192.168.168.102_16_04_37_2089dfb0-9534-4ba8-b2ba-77492c76c1a5_48.jpg: 384x640 2 hands, 1 head, 1 person, 4 products, 4.1ms
image 3/10 c:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\images\192.168.168.102_16_16_55_a63f5bd9-e5f3-4cbd-b35f-2f2807ceb51d_720.jpg: 384x640 5 hands, 1 head, 4 persons, 5 products, 4.0ms
image 4/10 c:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\images\192.168.168.102_17_54_59_a5703893-e236-4d53-b280-763711a77f15_44.jpg: 384x640 2 hands, 1 head, 1 person, 7 products, 8.0ms
image 5/10 c:\Users\ilias\Documents\Computer vision\euroshop\euroshop\euroshop_dataset\images\192.168.8.102_09_49_04_91dd2972-a8cf-4e5